# Sight Lines computation

In [1]:
import arcpy

arcpy.env.workspace = "C:/Users/g_filo01/sciebo/Scripts/Image of the City/Outputs/tmp"
env = arcpy.env.workspace
arcpy.CheckOutExtension("3D")
arcpy.env.overwriteOutput = True

In [ ]:
# output folder from which loading
city_name = 'London'
load_folder = "C:/Users/g_filo01/sciebo/Scripts/Image of the City/Outputs/"+city_name

In [2]:
observer_points = load_folder+"/"+city_name+"_nodes.shp"

buildings = "C:/Users/g_filo01/sciebo/GIS Data/London/Buildings/London_buildingsBase.shp"
buildings3d = "buildings3d"

sightlines_file = city_name+"_sightlines.shp"
height_observer = "height"
heigth_buildings = "height"
direction = "NOT_OUTPUT_THE_DIRECTION"

In [ ]:
aprx = arcpy.mp.ArcGISProject("C:/Users/g_filo01/sciebo/Scripts/ArcGis/empty.aprx")
buildings_layer = arcpy.MakeFeatureLayer_management(buildings, "as_layer").getOutput(0)
layer3d = arcpy.FeatureTo3DByAttribute_3d(buildings_layer, "layer3d", "height").getOutput(0)
temp = arcpy.AddZInformation_3d(layer3d, 'Z_MAX', 'NO_FILTER').getOutput(0)
# tempII = arcpy.AddZInformation_3d(temp, 'Z_MAX', 'NO_FILTER').getOutput(0)

In [ ]:
layer = arcpy.MakeFeatureLayer_management(temp, "tmp_layer").getOutput(0)

In [ ]:
m = aprx.listMaps()[0]
m.addLayer(layer)

for ma in aprx.listMaps():
    for lyr in ma.listLayers():
        print(lyr)
        lyr.extrusion("ABSOLUTE_HEIGHT", "Z_MAX")

In [ ]:
# obstructions = arcpy.Layer3DToFeatureClass_3d(m.listLayers()[0],
#                                               "obs", None, "ENABLE_COLORS_AND_TEXTURES").getOutput(0)

In [ ]:
# layer = arcpy.MakeFeatureLayer_management(obstructions, "obstructions_layer").getOutput(0)
# m.addLayer(layer)

In [ ]:
# for ma in aprx.listMaps():
#     for lyr in ma.listLayers():
#         print(lyr)
#         lyr.extrusion("ABSOLUTE_HEIGHT", "Z_MAX")

In [ ]:
aprx.saveACopy(env+"/copy.aprx")

In [5]:
geoDB = "C:/Users/g_filo01/sciebo/Scripts/ArcGis/GeoDB.gdb"
obstructions = geoDB+"/London_obstructions"

In [ ]:
arcpy.ddd.ConstructSightLines(observer_points, buildings, sightlines_file, 
                              height_observer, heigth_buildings, None, 30, direction)

In [ ]:
with arcpy.da.UpdateCursor(sightlines_file, "Shape_Length") as cursor:
    for row in cursor:
        if row[0] < 100:
            cursor.deleteRow()

In [ ]:
arcpy.ddd.Intervisibility(sightlines_file, obstructions = obstructions, visible_field="Visibility")

In [ ]:
arcpy.FeatureClassToShapefile_conversion(sightlines_file, arcpy.env.workspace)

In [8]:
with arcpy.da.UpdateCursor(sightlines_file, "Visibility") as cursor:
    for row in cursor:
        if row[0] == 0: 
            cursor.deleteRow()

In [10]:
arcpy.JoinField_management(sightlines_file, "OID_OBSERV", observer_points, "FID", "nodeID")
arcpy.JoinField_management(sightlines_file, "OID_TARGET", buildings, "FID", "buildingID")
arcpy.DeleteField_management(sightlines_file, ["OID_OBSERV", "OID_TARGET"])

<Result 'C:/Users/g_filo01/sciebo/Scripts/ArcGis/GeoDB.gdb/all_sight_lines'>

In [12]:
arcpy.FeatureClassToShapefile_conversion(sightlines_file, arcpy.env.workspace)

<Result 'C:\\Users\\g_filo01\\sciebo\\Scripts\\Image of the City\\Outputs\\tmp'>

# Visibility

In [ ]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import geopandas as gpd
import functools

from scipy import sparse
from scipy.sparse import linalg
import time
from shapely.geometry import Point, LineString, Polygon, MultiPolygon, mapping
from math import sqrt
from shapely.ops import cascaded_union
pd.set_option('precision', 5)

In [ ]:
file = 'Outputs/tmp/'+city_name+'_sight_lines.shp'
folder_ouptut ='Outputs/'+city_name+'/'
sight_lines = gpd.read_file(file)

In [ ]:
avg = (sight_lines[['buildingID', 'length']].groupby(['buildingID'], 
                                            as_index=False)['length'].mean())
avg.rename(columns={'length': 'mean_length'}, inplace=True)
avg.head()

In [ ]:
count =(sight_lines[['buildingID', 'length']].groupby(['buildingID'], 
                                            as_index=False)['length'].count())
count.rename(columns={'length': 'n_lines'}, inplace=True)
count.head()

In [ ]:
copy = sight_lines.set_index('buildingID')

distant = copy.groupby('buildingID').agg(lambda copy: copy.values[copy['length'].values.argmax()])
distant = distant.reset_index()
distant.head()

In [ ]:
visibility = pd.merge(distant, avg, left_on='buildingID', right_on='buildingID')
visibility.drop(['DIST_ALONG','Visibility', 'geometry'],axis=1, inplace=True)
visibility.rename(columns={'length':'distance','mean_length':'mean_distance'}, inplace=True)
visibility.to_csv('Outputs/tmp/visibility.csv')